In [1]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd 
import splitfolders

from keras.preprocessing.image import ImageDataGenerator

from keras import models
from keras import layers 
from keras import optimizers
from sklearn.dummy import DummyClassifier
from sklearn.metrics import recall_score
from keras import metrics
from tensorflow.keras.wrappers import scikit_learn
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

### Pre-processing

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=3747,
        color_mode='grayscale',
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 3747 images belonging to 2 classes.


In [5]:
validation_generator = val_datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=936,
                                                        color_mode='grayscale',
                                                        class_mode='binary')

Found 936 images belonging to 2 classes.


In [6]:
test_generator = test_datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=1173,
                                                  color_mode='grayscale',
                                                  class_mode='binary')

Found 1173 images belonging to 2 classes.


In [7]:
#Creating the augumented data

aug_train_images = ImageDataGenerator(rotation_range=30, 
                                   width_shift_range=0.25, 
                                   height_shift_range=0.25, 
                                   shear_range=0.25, 
                                   zoom_range=0.25, 
                                   horizontal_flip=True,
                                   vertical_flip=True)

train_aug = aug_train_images.flow_from_directory('Data_Split/train',
                                                  target_size=(150, 150),
                                                  batch_size=3747,
                                                  color_mode='grayscale',
                                                  class_mode='binary')

Found 3747 images belonging to 2 classes.


In [8]:
train_data, train_labels = next (train_generator)
test_data, test_labels = next (test_generator)
val_data, val_labels = next (validation_generator)

In [ ]:
train_data = train_data.reshape(train_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)
val_data = val_data.reshape(val_data.shape[0], -1)
train_data.shape

#### Dummy (baseline model)

In [31]:
dummy_model =  DummyClassifier(strategy='most_frequent')
dummy_model.fit(train_data, train_labels)

DummyClassifier(strategy='most_frequent')

In [32]:
y_preds_dummy = dummy_model.predict(val_data)

In [33]:
dummy_acc = dummy_model.score(val_data, val_labels)
dummy_rec = recall_score(val_labels,y_preds_dummy)
print(dummy_acc)
print(dummy_rec)

0.7297008547008547
1.0


In [35]:
train_data.shape

(3747, 150, 150, 1)

(3747, 22500)

#### Simple Model 1

In [46]:
simple_model = models.Sequential()

In [47]:
simple_model.add(layers.Dense(12, activation='relu', input_shape=(22500,)))
simple_model.add(layers.Dense(1, activation='sigmoid'))                       

In [48]:
simple_model.compile(optimizer='SGD',
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [49]:
simple_model_fit = simple_model.fit(train_data, train_labels, epochs=10, 
                                    batch_size=32, validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 2s 13ms/step - loss: 0.6977 - accuracy: 0.7246 - precision_1: 0.7291 - recall_1: 0.9905 - val_loss: 0.6457 - val_accuracy: 0.7297 - val_precision_1: 0.7297 - val_recall_1: 1.0000
Epoch 2/10
118/118 [==============================] - 1s 7ms/step - loss: 0.6317 - accuracy: 0.7297 - precision_1: 0.7297 - recall_1: 1.0000 - val_loss: 0.6197 - val_accuracy: 0.7297 - val_precision_1: 0.7297 - val_recall_1: 1.0000
Epoch 3/10
118/118 [==============================] - 1s 8ms/step - loss: 0.6116 - accuracy: 0.7297 - precision_1: 0.7297 - recall_1: 1.0000 - val_loss: 0.6046 - val_accuracy: 0.7297 - val_precision_1: 0.7297 - val_recall_1: 1.0000
Epoch 4/10
118/118 [==============================] - 1s 8ms/step - loss: 0.5984 - accuracy: 0.7297 - precision_1: 0.7297 - recall_1: 1.0000 - val_loss: 0.5948 - val_accuracy: 0.7297 - val_precision_1: 0.7297 - val_recall_1: 1.0000
Epoch 5/10
118/118 [==============================] - 1s 8ms/step - los

##### Simple Model GridSearch

In [69]:
def create_model():
    #create model
    model = models.Sequential()
    model.add(Dense(12, activation='relu', input_shape=(22500,)))
    model.add(Dense(1, activation='sigmoid'))
    #Compile model
    model.compile(optimizer='SGD', loss='binary_crossentropy',
                  metrics=['accuracy', metrics.Precision(), metrics.Recall()])
    return model
  

In [70]:
keras_model = scikit_learn.KerasClassifier(create_model, verbose=1)
batch_size = [10, 20]
epochs = [10, 25]
param_grid = dict(batch_size=batch_size, epochs=epochs)


In [ ]:
seed = 42
tf.random.set_seed(seed)

In [71]:
grid = GridSearchCV(estimator= keras_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(train_data, train_labels)

Epoch 1/10
300/300 [==============================] - 2s 8ms/step - loss: 0.4954 - accuracy: 0.7758 - precision_2: 0.7770 - recall_2: 0.9697
Epoch 2/10
300/300 [==============================] - 2s 5ms/step - loss: 0.3526 - accuracy: 0.8545 - precision_2: 0.8871 - recall_2: 0.9164
Epoch 3/10
300/300 [==============================] - 2s 5ms/step - loss: 0.3164 - accuracy: 0.8889 - precision_2: 0.9135 - recall_2: 0.9357
Epoch 4/10
300/300 [==============================] - 2s 5ms/step - loss: 0.2908 - accuracy: 0.8976 - precision_2: 0.9208 - recall_2: 0.9399A: 0s - loss: 0.3000 - accuracy: 0.8964 - precision_2: 0.9199 - recall_2: 
Epoch 5/10
300/300 [==============================] - 2s 5ms/step - loss: 0.2584 - accuracy: 0.9142 - precision_2: 0.9380 - recall_2: 0.9444
Epoch 6/10
300/300 [==============================] - 2s 6ms/step - loss: 0.2432 - accuracy: 0.9209 - precision_2: 0.9446 - recall_2: 0.9467
Epoch 7/10
300/300 [==============================] - 2s 6ms/step - loss: 0.2172

300/300 [==============================] - 2s 5ms/step - loss: 0.5827 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 5/25
300/300 [==============================] - 2s 7ms/step - loss: 0.5818 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 6/25
300/300 [==============================] - 2s 5ms/step - loss: 0.5816 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 7/25
300/300 [==============================] - 2s 6ms/step - loss: 0.5816 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 8/25
300/300 [==============================] - 2s 5ms/step - loss: 0.5815 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 9/25
300/300 [==============================] - 2s 6ms/step - loss: 0.5815 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 10/25
300/300 [==============================] - 2s 6ms/step - loss: 0.5815 - accuracy: 0.7318 - precision_9: 0.7318 - recall_9: 1.0000
Epoch 11/25
300/300 [==

150/150 [==============================] - 1s 4ms/step - loss: 0.2978 - accuracy: 0.9029 - precision_15: 0.9238 - recall_15: 0.9438
Epoch 7/10
150/150 [==============================] - 1s 6ms/step - loss: 0.2750 - accuracy: 0.9083 - precision_15: 0.9294 - recall_15: 0.9452
Epoch 8/10
150/150 [==============================] - 1s 7ms/step - loss: 0.2723 - accuracy: 0.9096 - precision_15: 0.9338 - recall_15: 0.9420
Epoch 9/10
150/150 [==============================] - 1s 8ms/step - loss: 0.2653 - accuracy: 0.9116 - precision_15: 0.9360 - recall_15: 0.9424
Epoch 10/10
38/38 [==============================] - 0s 3ms/step - loss: 0.2023 - accuracy: 0.9453 - precision_15: 0.9627 - recall_15: 0.9644
Epoch 1/10
150/150 [==============================] - 1s 4ms/step - loss: 0.4903 - accuracy: 0.7785 - precision_16: 0.7801 - recall_16: 0.9723
Epoch 2/10
150/150 [==============================] - 1s 4ms/step - loss: 0.3618 - accuracy: 0.8396 - precision_16: 0.8675 - recall_16: 0.9223A: 0s - loss

Epoch 15/25
150/150 [==============================] - 1s 4ms/step - loss: 0.2391 - accuracy: 0.9223 - precision_18: 0.9446 - recall_18: 0.9494
Epoch 16/25
150/150 [==============================] - 1s 4ms/step - loss: 0.2274 - accuracy: 0.9236 - precision_18: 0.9443 - recall_18: 0.9516
Epoch 17/25
150/150 [==============================] - 1s 4ms/step - loss: 0.2139 - accuracy: 0.9286 - precision_18: 0.9447 - recall_18: 0.9585
Epoch 18/25
150/150 [==============================] - 1s 4ms/step - loss: 0.2171 - accuracy: 0.9266 - precision_18: 0.9470 - recall_18: 0.9530
Epoch 19/25
150/150 [==============================] - 1s 4ms/step - loss: 0.2138 - accuracy: 0.9303 - precision_18: 0.9480 - recall_18: 0.9571
Epoch 20/25
150/150 [==============================] - 1s 5ms/step - loss: 0.2151 - accuracy: 0.9249 - precision_18: 0.9456 - recall_18: 0.9521
Epoch 21/25
150/150 [==============================] - 1s 5ms/step - loss: 0.2069 - accuracy: 0.9276 - precision_18: 0.9450 - recall_18:

### Convolutional Neural Network

In [87]:
train_data, train_labels = next (train_generator)
test_data, test_labels = next (test_generator)
val_data, val_labels = next (validation_generator)

In [88]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150,150,1)))
cnn_model.add(layers.MaxPooling2D((2,2)))
cnn_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn_model.add(layers.MaxPooling2D((2,2)))
cnn_model.add(layers.Flatten())
cnn_model.add(Dense(12, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))              

In [89]:
cnn_model.compile(optimizer='SGD',
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [90]:
cnn_model_fit = cnn_model.fit(train_data, train_labels, epochs=10, 
                                    batch_size=32, validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 170s 1s/step - loss: 0.5810 - accuracy: 0.7254 - precision_24: 0.7295 - recall_24: 0.9912 - val_loss: 0.6092 - val_accuracy: 0.7981 - val_precision_24: 0.7865 - val_recall_24: 0.9927
Epoch 2/10
118/118 [==============================] - 146s 1s/step - loss: 0.4824 - accuracy: 0.7654 - precision_24: 0.7624 - recall_24: 0.9857 - val_loss: 0.4594 - val_accuracy: 0.7415 - val_precision_24: 0.7384 - val_recall_24: 1.0000
Epoch 3/10
118/118 [==============================] - 140s 1s/step - loss: 0.3821 - accuracy: 0.8348 - precision_24: 0.8562 - recall_24: 0.9298 - val_loss: 1.2761 - val_accuracy: 0.3504 - val_precision_24: 1.0000 - val_recall_24: 0.1098
Epoch 4/10
118/118 [==============================] - 139s 1s/step - loss: 0.3178 - accuracy: 0.8692 - precision_24: 0.8953 - recall_24: 0.9294 - val_loss: 0.6342 - val_accuracy: 0.7585 - val_precision_24: 0.7514 - val_recall_24: 1.0000
Epoch 5/10
118/118 [==============================] 

KeyboardInterrupt: 

In [80]:
def create_model():
    #create model
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150,150,1)))
    cnn_model.add(layers.MaxPooling2D((2,2)))
    cnn_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
    cnn_model.add(layers.MaxPooling2D((2,2)))
    cnn_model.add(layers.Flatten())
    cnn_model.add(Dense(12, activation='relu'))
    cnn_model.add(Dense(1, activation='sigmoid')) 
    #Compile model
    model.compile(optimizer='SGD', loss='binary_crossentropy',
                  metrics=['accuracy', metrics.Precision(), metrics.Recall()])
    return model

In [81]:
cnn_model = scikit_learn.KerasClassifier(create_model, verbose=1)
batch_size = [10, 20]
epochs = [10, 25]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [ ]:
grid = GridSearchCV(estimator= cnn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(train_data, train_labels)

In [9]:
cnn1_model = models.Sequential()
cnn1_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150,150,1)))
cnn1_model.add(layers.MaxPooling2D((2,2)))
cnn1_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn1_model.add(layers.MaxPooling2D((2,2)))
cnn1_model.add(layers.Flatten())
cnn1_model.add(Dense(12, activation='relu'))
cnn1_model.add(Dense(1, activation='sigmoid')) 
    #Compile model
cnn1_model.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['accuracy', metrics.Precision(), metrics.Recall()])

cnn1_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 12)                4

In [10]:
cnn1_model_fit = cnn1_model.fit(train_data, 
                                train_labels, 
                                epochs=10, 
                                batch_size=32, 
                                validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 275s 2s/step - loss: 0.4258 - accuracy: 0.7886 - precision: 0.7859 - recall: 0.9762 - val_loss: 0.2976 - val_accuracy: 0.9038 - val_precision: 0.8980 - val_recall: 0.9795
Epoch 2/10
118/118 [==============================] - 255s 2s/step - loss: 0.3019 - accuracy: 0.8983 - precision: 0.8939 - recall: 0.9766 - val_loss: 0.3211 - val_accuracy: 0.9167 - val_precision: 0.9871 - val_recall: 0.8975
Epoch 3/10
118/118 [==============================] - 277s 2s/step - loss: 0.2629 - accuracy: 0.9338 - precision: 0.9343 - recall: 0.9781 - val_loss: 0.2333 - val_accuracy: 0.9455 - val_precision: 0.9514 - val_recall: 0.9751
Epoch 4/10
118/118 [==============================] - 371s 3s/step - loss: 0.2443 - accuracy: 0.9397 - precision: 0.9475 - recall: 0.9711 - val_loss: 0.2188 - val_accuracy: 0.9498 - val_precision: 0.9663 - val_recall: 0.9649
Epoch 5/10
118/118 [==============================] - 264s 2s/step - loss: 0.2286 - accuracy: 0.9469

In [11]:
cnn2_model = models.Sequential()
cnn2_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150,150,1)))
cnn2_model.add(layers.MaxPooling2D((2,2)))
cnn2_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn2_model.add(layers.MaxPooling2D((2,2)))
cnn2_model.add(layers.Flatten())
cnn2_model.add(Dense(12, activation='relu'))
cnn2_model.add(Dense(1, activation='sigmoid')) 
    #Compile model
cnn2_model.compile(optimizer='rmsprop', loss='binary_crossentropy',
                  metrics=['accuracy', metrics.Precision(), metrics.Recall()])

cnn2_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 41472)             0         
                                                                 
 dense_2 (Dense)             (None, 12)               

In [12]:
cnn2_model_fit = cnn2_model.fit(train_data, 
                                train_labels, 
                                epochs=10, 
                                batch_size=32, 
                                validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 305s 3s/step - loss: 0.4296 - accuracy: 0.8332 - precision_1: 0.8583 - recall_1: 0.9239 - val_loss: 0.1852 - val_accuracy: 0.9209 - val_precision_1: 0.9607 - val_recall_1: 0.9297
Epoch 2/10
118/118 [==============================] - 289s 2s/step - loss: 0.2052 - accuracy: 0.9239 - precision_1: 0.9406 - recall_1: 0.9561 - val_loss: 0.1311 - val_accuracy: 0.9466 - val_precision_1: 0.9580 - val_recall_1: 0.9693
Epoch 3/10
118/118 [==============================] - 292s 2s/step - loss: 0.1634 - accuracy: 0.9458 - precision_1: 0.9600 - recall_1: 0.9660 - val_loss: 0.1596 - val_accuracy: 0.9423 - val_precision_1: 0.9831 - val_recall_1: 0.9370
Epoch 4/10
118/118 [==============================] - 289s 2s/step - loss: 0.1321 - accuracy: 0.9538 - precision_1: 0.9655 - recall_1: 0.9715 - val_loss: 0.1108 - val_accuracy: 0.9519 - val_precision_1: 0.9691 - val_recall_1: 0.9649
Epoch 5/10
118/118 [==============================] - 297s 3s/step -

In [13]:
cnn3_model = models.Sequential()
cnn3_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150,150,1)))
cnn3_model.add(layers.MaxPooling2D((2,2)))
cnn3_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn3_model.add(layers.MaxPooling2D((2,2)))
cnn3_model.add(layers.Flatten())
cnn3_model.add(Dense(12, activation='relu'))
cnn3_model.add(Dense(1, activation='sigmoid')) 
    #Compile model
cnn3_model.compile(optimizer='adadelta', loss='binary_crossentropy',
                  metrics=['accuracy', metrics.Precision(), metrics.Recall()])

cnn3_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 41472)             0         
                                                                 
 dense_4 (Dense)             (None, 12)               

In [14]:
cnn3_model_fit = cnn3_model.fit(train_data, 
                                train_labels, 
                                epochs=10, 
                                batch_size=32, 
                                validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 302s 3s/step - loss: 0.6590 - accuracy: 0.7256 - precision_2: 0.7294 - recall_2: 0.9920 - val_loss: 0.6358 - val_accuracy: 0.7297 - val_precision_2: 0.7297 - val_recall_2: 1.0000
Epoch 2/10
118/118 [==============================] - 294s 2s/step - loss: 0.6223 - accuracy: 0.7297 - precision_2: 0.7297 - recall_2: 1.0000 - val_loss: 0.6094 - val_accuracy: 0.7297 - val_precision_2: 0.7297 - val_recall_2: 1.0000
Epoch 3/10
118/118 [==============================] - 285s 2s/step - loss: 0.6033 - accuracy: 0.7297 - precision_2: 0.7297 - recall_2: 1.0000 - val_loss: 0.5963 - val_accuracy: 0.7297 - val_precision_2: 0.7297 - val_recall_2: 1.0000
Epoch 4/10
118/118 [==============================] - 296s 3s/step - loss: 0.5947 - accuracy: 0.7297 - precision_2: 0.7297 - recall_2: 1.0000 - val_loss: 0.5907 - val_accuracy: 0.7297 - val_precision_2: 0.7297 - val_recall_2: 1.0000
Epoch 5/10
118/118 [==============================] - 261s 2s/step -

In [18]:
cnn4_model = models.Sequential()
cnn4_model.add(layers.Conv2D(64, (3, 3), activation='relu', 
                            input_shape=(150,150,1)))
cnn4_model.add(layers.MaxPooling2D((2,2)))
cnn4_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn4_model.add(layers.MaxPooling2D((2,2)))
cnn4_model.add(layers.Flatten())
cnn4_model.add(Dense(16, activation='relu'))
cnn4_model.add(Dense(1, activation='sigmoid')) 

cnn4_model.compile(optimizer="adam",
                        loss='binary_crossentropy',
                        metrics=['accuracy', metrics.Precision(), metrics.Recall()])

cnn4_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 41472)             0         
                                                                 
 dense_8 (Dense)             (None, 16)               

In [19]:
cnn4_model_fit = cnn4_model.fit(train_data, 
                                train_labels, 
                                epochs=10, 
                                batch_size=32, 
                                validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 147s 1s/step - loss: 0.3201 - accuracy: 0.8647 - precision_4: 0.8812 - recall_4: 0.9415 - val_loss: 0.1827 - val_accuracy: 0.9241 - val_precision_4: 0.9513 - val_recall_4: 0.9444
Epoch 2/10
118/118 [==============================] - 134s 1s/step - loss: 0.1530 - accuracy: 0.9434 - precision_4: 0.9546 - recall_4: 0.9685 - val_loss: 0.1492 - val_accuracy: 0.9380 - val_precision_4: 0.9358 - val_recall_4: 0.9824
Epoch 3/10
118/118 [==============================] - 143s 1s/step - loss: 0.1442 - accuracy: 0.9440 - precision_4: 0.9559 - recall_4: 0.9678 - val_loss: 0.5587 - val_accuracy: 0.8077 - val_precision_4: 0.9980 - val_recall_4: 0.7379
Epoch 4/10
118/118 [==============================] - 170s 1s/step - loss: 0.1426 - accuracy: 0.9482 - precision_4: 0.9611 - recall_4: 0.9682 - val_loss: 0.1189 - val_accuracy: 0.9530 - val_precision_4: 0.9624 - val_recall_4: 0.9736
Epoch 5/10
118/118 [==============================] - 146s 1s/step -

In [20]:
cnn5_model = models.Sequential()
cnn5_model.add(layers.Conv2D(64, (3, 3), activation='relu', 
                            input_shape=(150,150,1)))
cnn5_model.add(layers.MaxPooling2D((2,2)))
cnn5_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn5_model.add(layers.MaxPooling2D((2,2)))
cnn5_model.add(layers.Flatten())
cnn5_model.add(Dense(16, activation='relu'))
cnn5_model.add(Dense(1, activation='sigmoid')) 

cnn5_model.compile(optimizer="adam",
                        loss='binary_crossentropy',
                        metrics=['accuracy', metrics.Precision(), metrics.Recall()])

cnn5_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 41472)             0         
                                                                 
 dense_10 (Dense)            (None, 16)               

In [21]:
cnn5_model_fit = cnn5_model.fit(train_data, 
                                train_labels, 
                                epochs=25, 
                                batch_size=32, 
                                validation_data= (val_data, val_labels), verbose=1)

Epoch 1/25
118/118 [==============================] - 151s 1s/step - loss: 0.2948 - accuracy: 0.8732 - precision_5: 0.8838 - recall_5: 0.9514 - val_loss: 0.2613 - val_accuracy: 0.8985 - val_precision_5: 0.8828 - val_recall_5: 0.9927
Epoch 2/25
118/118 [==============================] - 152s 1s/step - loss: 0.1669 - accuracy: 0.9376 - precision_5: 0.9522 - recall_5: 0.9627 - val_loss: 0.1538 - val_accuracy: 0.9380 - val_precision_5: 0.9845 - val_recall_5: 0.9297
Epoch 3/25
118/118 [==============================] - 184s 2s/step - loss: 0.1435 - accuracy: 0.9437 - precision_5: 0.9582 - recall_5: 0.9649 - val_loss: 0.1341 - val_accuracy: 0.9444 - val_precision_5: 0.9788 - val_recall_5: 0.9444
Epoch 4/25
118/118 [==============================] - 158s 1s/step - loss: 0.1284 - accuracy: 0.9522 - precision_5: 0.9671 - recall_5: 0.9674 - val_loss: 0.1316 - val_accuracy: 0.9498 - val_precision_5: 0.9466 - val_recall_5: 0.9868
Epoch 5/25
118/118 [==============================] - 153s 1s/step -

In [ ]:
cnn6_model = models.Sequential()
cnn6_model.add(layers.Conv2D(64, (3, 3), activation='relu', 
                            input_shape=(150,150,1)))
cnn6_model.add(layers.MaxPooling2D((2,2)))
cnn6_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn6_model.add(layers.MaxPooling2D((2,2)))
cnn6_model.add(layers.Flatten())
cnn6_model.add(Dense(16, activation='relu'))
cnn6_model.add(Dense(16, activation='relu'))
cnn6_model.add(Dense(4, activation='sigmoid')) 

cnn6_model.compile(optimizer="adam",
                        loss='binary_crossentropy',
                        metrics=['accuracy', metrics.Precision(), metrics.Recall()])

cnn6_model.summary()

cnn6_model_fit = cnn6_model.fit(train_data, 
                                train_labels, 
                                epochs=10, 
                                batch_size=32, 
                                validation_data= (val_data, val_labels), verbose=1)